In [ ]:
import ctypes
import numpy as np
import time
import cv2
import gc
import os
import zlib
from _retro import RetroEmulator

core = os.path.abspath("../cores/pcsx2_libretro.so")
game = os.path.abspath("../../roms/Granturismo3.iso")

env = RetroEmulator()
env2 = RetroEmulator()
env.init(core, game)
# env2.init(core, game)

env.run()
env2.run()

def get_frame(_env, width=640, height=480):
    buffer = (ctypes.c_uint8 * (width * height * 3))()
    _env.get_frame(buffer, width, height)
    
    img = np.frombuffer(buffer, dtype=np.uint8)
    return img.reshape((height, width, 3))[::-1]


count = 0

def reset():
    env.set_button_mask(np.zeros([16], np.uint8), 0)
    env.reset()

initial_state = None

reset()

key = 0
key_count = 0

while True:
    action = np.zeros(16, dtype=np.uint8)

    if count % 100 == 0 and count > 0:
        action[3] = 1
        action[0] = 1
    
    env.set_button_mask(action, 0)
    env.run()
    env2.run()
    
    img = get_frame(env)
    img2 = get_frame(env2)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2BGR)
    cv2.imshow("env", img)
    cv2.waitKey(1)

    cv2.imshow("env2", img2)
    cv2.waitKey(1)

    count += 1

    # break

    if count % 1000 == 0:
        env.reset()

    # if initial_state != None and count % 500 == 0:
    #     # env.set_state(initial_state)
    #     initial_state = None
        
    # if count % 1000 == 0:
    #     initial_state = env.get_state()
    #     # env.reset()
    #     reset()
    #     print("save state")
    #     # lib.save_state("game.state".encode("utf-8"))
        # break